## This is a cleaned up workbook working through the development of scraping functions for Jeopardy games (individual shows).  The .py function file has been updated so the first part is slightly out of date, but can be used to generate the overall pattern of how the acquisition works. 

----

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re

### Create Base Soup Object - Single Page Run

In [2]:
r = requests.get('https://j-archive.com/showgame.php?game_id=7409')
soup = bs(r.content )

### Get Show Info

In [3]:
show_info = soup.find('div', attrs = {'id':'game_title'}).text

In [4]:
show_num = re.findall(r'\d+', show_info)[0]
show_yr = re.findall(r'\d+', show_info)[2][2:]

### Identify Daily Doubles

In [5]:
indexer = 1
dd_list = []
for thing in soup.find_all('td', class_ = 'clue'):
    row = {}
    row['q_index'] = indexer
    indexer += 1
    if thing.find('td', class_ = 'clue_value_daily_double'):
        row['is_DD'] = 1
    else:
        row['is_DD'] = 0
    dd_list.append(row)
df_dd = pd.DataFrame(dd_list)

### Identify Answers and Triple Stumpers

In [6]:
answers = []
counter = 1

for clue in soup.find_all('td', class_ = 'clue'):
    answer = {}
    if counter < 61:
        answer['q_index'] = counter
        ans = clue.find('div', onmouseover = True).get('onmouseover')
        cls = bs(ans)
        answer['answer'] = cls.find('em').text
        if 'Triple Stumper' in ans:
            answer['is_stumper'] = 1
        else:
            answer['is_stumper'] = 0
        answers.append(answer)
        counter += 1
df_ans = pd.DataFrame(answers)

### Create Question List

In [7]:
j_value = ['200','400','600','800','1000'] #should these be ints?
dj_value = ['400','800','1200','1600','2000'] #SHould these be ints?

In [8]:
# Jeopardy Categories
j_category_list = soup.find('div', attrs = {'id':'jeopardy_round'}).find_all('td', attrs = {'class':'category_name'})
j_categories = []
for category in j_category_list:
    j_categories.append(category.get_text())

In [9]:
# Jeopardy:
questions = []

for i in range(6):
    for j in range(5):
        question = {}
        
        question['show_num'] = show_num
        question['show_yr'] = show_yr
        
        question['round'] = 'Jeopardy'
        
        question['category'] = j_categories[i]
         
        question['value'] = j_value[j]
        
        clue = f'clue_J_{i+1}_{j+1}' 
        question['clue'] = soup.find('td', attrs = {'id':clue}).text
        
        question['q_index'] = (i + 1) + (6* j)
        
        questions.append(question)
df_j_qs = pd.DataFrame(questions)

In [10]:
# Double Jeopardy Categories
dj_category_list = soup.find('div', attrs = {'id':'double_jeopardy_round'}).find_all('td', attrs = {'class':'category_name'})
dj_categories = []
for category in dj_category_list:
    dj_categories.append(category.get_text())

In [11]:
# Double Jeopardy
questions = []

for i in range(6):
    for j in range(5):
        question = {}
        question['q_index'] = 30 + (i + 1) + (6* j)
        question['show_num'] = show_num
        question['show_yr'] = show_yr
        
        question['round'] = 'Double Jeopardy'
        
        question['category'] = dj_categories[i]
         
        question['value'] = dj_value[j]
        
        clue = f'clue_DJ_{i+1}_{j+1}' 
        try:
                question['clue'] = soup.find('td', attrs = {'id':clue}).text
        except:
            continue        
        
        questions.append(question)
df_dj_qs = pd.DataFrame(questions)

In [12]:
# Final Jeopardy

#q_index is unnecessary as this final jeopardy will be assigned after other dfs are combined

fj = soup.find('table', class_ = 'final_round')

f_j = {}
f_j['show_num'] = show_num
f_j['show_yr'] = show_yr
f_j['round'] = 'Final Jeopardy'
f_j['value'] = 'FJ'
f_j['is_DD'] = 0

f_j['category'] = fj.find('td', class_ = 'category_name').text

f_j['clue'] = fj.find('td', attrs = {'id':'clue_FJ'}).text

ans = fj.find('div', onmouseover = True).get('onmouseover')
cls = bs(ans)
f_j['answer'] = cls.find('em').text
if 'Triple Stumper' in ans:
    f_j['is_stumper'] = 1
else:
    f_j['is_stumper'] = 0

df_fj = pd.DataFrame([f_j])

### Combine Dataframes

In [13]:
df_js = pd.concat([df_j_qs, df_dj_qs]).reset_index()

In [14]:
df_game = df_js.merge(df_dd, on='q_index').merge(df_ans, on = 'q_index').drop(columns=['index','q_index'])
df_game = pd.concat([df_game,df_fj]).reset_index()
df_game

,index,show_num,show_yr,round,category,value,clue,is_DD,answer,is_stumper
0,0,8681,22,Jeopardy,GLACIERS,200,Iceland's vast Vatna Glacier is dotted with th...,0,volcanoes,0
1,1,8681,22,Jeopardy,GLACIERS,400,Tasman Lake in this country is one of the few ...,0,New Zealand,0
2,2,8681,22,Jeopardy,GLACIERS,600,99% of the world's glacial ice is located on A...,0,Greenland,0
3,3,8681,22,Jeopardy,GLACIERS,800,This glacier in Alaska is named for naturalist...,0,Muir (Muir Glacier),0
4,4,8681,22,Jeopardy,GLACIERS,1000,"When glaciers retreated in Norway, seawater fi...",0,fjords,0
...,...,...,...,...,...,...,...,...,...,...
56,56,8681,22,Double Jeopardy,FAMILIAL PHRASES,800,If things are going swimmingly for you in the ...,0,uncle,0
57,57,8681,22,Double Jeopardy,FAMILIAL PHRASES,1200,This rhyming phrase suggests women should prio...,0,sisters before misters,0
58,58,8681,22,Double Jeopardy,FAMILIAL PHRASES,1600,"As a verb, it means exempt from new legislation",1,grandfather,0
59,59,8681,22,Double Jeopardy,FAMILIAL PHRASES,2000,"This phrase from Shakespeare's ""Henry V"" now r...",0,band of brothers,0


### Add Level (Difficulty)

In [15]:
df_game['level'] =  np.where(df_game['is_DD'] == 1, 4, \
                    np.where(df_game['is_stumper'] == 1, 5, \
                    np.where(df_game['round'] == 'Final Jeopardy', 6, \
                    np.where((df_game['value'] == '200') | (df_game['value'] == '400'), 1, \
                    np.where((df_game['value'] == '600') | (df_game['value'] == '800') | (df_game['value'] == '1200'), 2, \
                    np.where((df_game['value'] == '1000') | (df_game['value'] == '1600') | (df_game['value'] == '2000'), 3, 'X''X')))))) 

In [16]:
df_game

,index,show_num,show_yr,round,category,value,clue,is_DD,answer,is_stumper,level
0,0,8681,22,Jeopardy,GLACIERS,200,Iceland's vast Vatna Glacier is dotted with th...,0,volcanoes,0,1
1,1,8681,22,Jeopardy,GLACIERS,400,Tasman Lake in this country is one of the few ...,0,New Zealand,0,1
2,2,8681,22,Jeopardy,GLACIERS,600,99% of the world's glacial ice is located on A...,0,Greenland,0,2
3,3,8681,22,Jeopardy,GLACIERS,800,This glacier in Alaska is named for naturalist...,0,Muir (Muir Glacier),0,2
4,4,8681,22,Jeopardy,GLACIERS,1000,"When glaciers retreated in Norway, seawater fi...",0,fjords,0,3
...,...,...,...,...,...,...,...,...,...,...,...
56,56,8681,22,Double Jeopardy,FAMILIAL PHRASES,800,If things are going swimmingly for you in the ...,0,uncle,0,2
57,57,8681,22,Double Jeopardy,FAMILIAL PHRASES,1200,This rhyming phrase suggests women should prio...,0,sisters before misters,0,2
58,58,8681,22,Double Jeopardy,FAMILIAL PHRASES,1600,"As a verb, it means exempt from new legislation",1,grandfather,0,4
59,59,8681,22,Double Jeopardy,FAMILIAL PHRASES,2000,"This phrase from Shakespeare's ""Henry V"" now r...",0,band of brothers,0,3


---------

# Functions

In [17]:
import acquire

In [18]:
test_url = 'https://j-archive.com/showgame.php?game_id=6833'

#### Testing Daily Doubles

In [19]:
df_dd = acquire.is_dd(test_url)
df_dd

,q_index,is_DD
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
56,57,0
57,58,0
58,59,0
59,60,0


*Can add error finders and messangers in here in the future, using things like ensuring DD value count = 3.*

#### Testing Answers and Triple Stumper Detector

In [20]:
df_ans = acquire.answers(test_url)
df_ans

,q_index,answer,is_stumper
0,1,the constitution,0
1,2,ragtime,0
2,3,a rod,0
3,4,the Pony Express,0
4,5,contact lenses,0
5,6,Mount Fuji,0
6,7,confidence,0
7,8,disco,0
8,9,bills,0
9,10,Attorney General,0


#### Jeopardy Questions

In [21]:
df_j_qs = acquire.j_qs(test_url)
df_j_qs

,q_index,show_num,show_yr,round,category,value,clue
0,1,8248,2020,Jeopardy,BRITISH GOVERNMENT,200,Acts of Parliament & other conventions & court...
1,7,8248,2020,Jeopardy,BRITISH GOVERNMENT,400,A vote of this in life means someone believes ...
2,13,8248,2020,Jeopardy,BRITISH GOVERNMENT,600,"Britain's standard V.A.T., short for this, is ..."
3,19,8248,2020,Jeopardy,BRITISH GOVERNMENT,800,"In your house, it's the 2-word room for your c..."
4,25,8248,2020,Jeopardy,BRITISH GOVERNMENT,1000,In 2006 in the British House of Lords the job ...
5,2,8248,2020,Jeopardy,MUSICAL STYLES,200,"Though lesser known than Joplin, African Ameri..."
6,8,8248,2020,Jeopardy,MUSICAL STYLES,400,"""The Hustle"" & ah, ah, ah, ah ""Stayin' Alive"" ..."
7,14,8248,2020,Jeopardy,MUSICAL STYLES,600,A 1992 N.Y. Times article asked how did this w...
8,20,8248,2020,Jeopardy,MUSICAL STYLES,800,Selena was named Female Vocalist of the Year 1...
9,26,8248,2020,Jeopardy,MUSICAL STYLES,1000,This composer and musician with the given name...


#### Double Jeopardy Questions

In [22]:
df_dj_qs = acquire.dj_qs(test_url)
df_dj_qs

,q_index,show_num,show_yr,round,category,value,clue
0,31,8248,2020,Double Jeopardy,THEATER TIME,400,Time to head to ancient Greece for the Sophocl...
1,37,8248,2020,Double Jeopardy,THEATER TIME,800,"The acclaimed ""Lehman Trilogy"" runs 3 hours-pl..."
2,43,8248,2020,Double Jeopardy,THEATER TIME,1200,"His ""Long Day's Journey into Night"" begins on ..."
3,49,8248,2020,Double Jeopardy,THEATER TIME,1600,The Tony-winning play named for this European ...
4,55,8248,2020,Double Jeopardy,THEATER TIME,2000,"No, we're certain this 2005 Pulitzer-winning d..."
5,32,8248,2020,Double Jeopardy,3 VOWELS IN A ROW,400,Adjective for one who's ready to take you to c...
6,38,8248,2020,Double Jeopardy,3 VOWELS IN A ROW,800,"Mais oui, it's a word for a fabulous mansion--..."
7,44,8248,2020,Double Jeopardy,3 VOWELS IN A ROW,1200,This mental disorder makes it seem like everyo...
8,50,8248,2020,Double Jeopardy,3 VOWELS IN A ROW,1600,Hunkpapa & Oglala are branches of this Native ...
9,56,8248,2020,Double Jeopardy,3 VOWELS IN A ROW,2000,"From the Latin for ""rest"", it's a religious se..."


#### Final Jeopardy

In [23]:
df_fj = acquire.fj(test_url)
df_fj

,show_num,show_yr,round,value,is_DD,category,clue,answer,is_stumper
0,8248,2020,Final Jeopardy,FJ,0,HISTORIC FIGURES,"In a 1912 telegram to his wife, he said, ""Am f...",Teddy Roosevelt,0


#### Show Dataframe

In [24]:
df_game = acquire.show_dataframe(df_dd, df_ans, df_j_qs, df_dj_qs, df_fj)
df_game

,show_num,show_yr,round,category,value,clue,is_DD,answer,is_stumper
0,8248,2020,Jeopardy,BRITISH GOVERNMENT,200,Acts of Parliament & other conventions & court...,0,the constitution,0
1,8248,2020,Jeopardy,BRITISH GOVERNMENT,400,A vote of this in life means someone believes ...,0,confidence,0
2,8248,2020,Jeopardy,BRITISH GOVERNMENT,600,"Britain's standard V.A.T., short for this, is ...",0,value added tax,0
3,8248,2020,Jeopardy,BRITISH GOVERNMENT,800,"In your house, it's the 2-word room for your c...",0,the home office,0
4,8248,2020,Jeopardy,BRITISH GOVERNMENT,1000,In 2006 in the British House of Lords the job ...,0,Lord High Chancellor,1
...,...,...,...,...,...,...,...,...,...
56,8248,2020,Double Jeopardy,FLOWERS ON THE WALL,800,(Jimmy of the Clue Crew presents by a display ...,0,the Crucifixion,0
57,8248,2020,Double Jeopardy,FLOWERS ON THE WALL,1200,"This woman said, ""I'll paint what I see--what ...",1,Georgia O\'Keeffe,0
58,8248,2020,Double Jeopardy,FLOWERS ON THE WALL,1600,"The last paintings by this ""Luncheon on the Gr...",0,Manet,0
59,8248,2020,Double Jeopardy,FLOWERS ON THE WALL,2000,Before his more geometric style of intersectin...,0,Mondrian,0


#### Add Level

In [25]:
df_game = acquire.add_level(df_game)
df_game

,show_num,show_yr,round,category,value,clue,is_DD,answer,is_stumper,level
0,8248,2020,Jeopardy,BRITISH GOVERNMENT,200,Acts of Parliament & other conventions & court...,0,the constitution,0,1
1,8248,2020,Jeopardy,BRITISH GOVERNMENT,400,A vote of this in life means someone believes ...,0,confidence,0,1
2,8248,2020,Jeopardy,BRITISH GOVERNMENT,600,"Britain's standard V.A.T., short for this, is ...",0,value added tax,0,2
3,8248,2020,Jeopardy,BRITISH GOVERNMENT,800,"In your house, it's the 2-word room for your c...",0,the home office,0,2
4,8248,2020,Jeopardy,BRITISH GOVERNMENT,1000,In 2006 in the British House of Lords the job ...,0,Lord High Chancellor,1,5
...,...,...,...,...,...,...,...,...,...,...
56,8248,2020,Double Jeopardy,FLOWERS ON THE WALL,800,(Jimmy of the Clue Crew presents by a display ...,0,the Crucifixion,0,2
57,8248,2020,Double Jeopardy,FLOWERS ON THE WALL,1200,"This woman said, ""I'll paint what I see--what ...",1,Georgia O\'Keeffe,0,4
58,8248,2020,Double Jeopardy,FLOWERS ON THE WALL,1600,"The last paintings by this ""Luncheon on the Gr...",0,Manet,0,3
59,8248,2020,Double Jeopardy,FLOWERS ON THE WALL,2000,Before his more geometric style of intersectin...,0,Mondrian,0,3


In [26]:
df_game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   show_num    61 non-null     object
 1   show_yr     61 non-null     object
 2   round       61 non-null     object
 3   category    61 non-null     object
 4   value       61 non-null     object
 5   clue        61 non-null     object
 6   is_DD       61 non-null     int64 
 7   answer      61 non-null     object
 8   is_stumper  61 non-null     int64 
 9   level       61 non-null     object
dtypes: int64(2), object(8)
memory usage: 4.9+ KB


#### Altogether Now

In [27]:
df_game2 = acquire.acquire_show(test_url)
df_game2.to_csv('show_8248.csv')

In [28]:
output = acquire.acquire_shows('showgame.php?game_id=7407')

Scraping page 1: game_id=7407
Clue Bank Size = 0
Scraping page 2: game_id=7409
Clue Bank Size = 61
Scraping page 3: game_id=7410
Clue Bank Size = 122
Scraping page 4: game_id=7411
Clue Bank Size = 183
Scraping page 5: game_id=7412
Clue Bank Size = 244
Scraping page 6: game_id=7413
Clue Bank Size = 305
Scraping page 7: game_id=7414
Clue Bank Size = 366
Scraping page 8: game_id=7415
Clue Bank Size = 427
Done Scraping!


In [29]:
output.sample(25)

,show_num,show_yr,round,category,value,clue,is_DD,answer,is_stumper,level
7,8681,2022,Jeopardy,HEARING RED,600,"For Peter Gabriel, this unusual title precipit...",0,"""Red Rain""",0,2
41,8686,2022,Double Jeopardy,TROPHY HUSBANDS,800,"When he won Best Director at Cannes for ""After...",0,Martin Scorsese,1,5
47,8683,2022,Double Jeopardy,HODGEPODGE,1200,"In 2021, the U.S. Postal Service issued new st...",0,the Day of the Dead (Día de los Muertos),0,2
4,8682,2022,Jeopardy,HUMAN ANATOMY,1000,These tiny air sacs in the lungs are where oxy...,0,alveoli,0,3
20,8684,2022,Jeopardy,TERRIBLE SUPERVILLAIN NAMES,200,The king can't move without being put in check...,0,a stalemate,0,1
0,8684,2022,Jeopardy,THE FOUNDING FATHERS,200,"Encyclopedia Britannica calls him ""The Foundin...",0,George Washington,0,1
53,8685,2022,Double Jeopardy,YOU WILL GET BETTER HERE,1600,The Trenton Psychiatric Hospital was originall...,0,an asylum,0,3
42,8686,2022,Double Jeopardy,TROPHY HUSBANDS,1200,Yvonne Molinaro married 2 winners of this grid...,0,the Heisman,0,2
36,8687,2022,Double Jeopardy,THE MOVIE'S DIRECTOR,800,"2003's ""Lost in Translation""",0,Sofia Coppola,0,2
17,8683,2022,Jeopardy,WELSH FOLK,600,A commander of the Order of the British Empire...,0,Catherine Zeta-Jones,1,5


In [30]:
output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488 entries, 0 to 60
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   show_num    488 non-null    object
 1   show_yr     488 non-null    object
 2   round       488 non-null    object
 3   category    488 non-null    object
 4   value       488 non-null    object
 5   clue        488 non-null    object
 6   is_DD       488 non-null    int64 
 7   answer      488 non-null    object
 8   is_stumper  488 non-null    int64 
 9   level       488 non-null    object
dtypes: int64(2), object(8)
memory usage: 41.9+ KB
